In [20]:
import pandas as pd
from io import StringIO
import os
from datetime import datetime, timedelta

from nst_scraper import nst_on_ice_scraper
from db_utils import append_player_ids

pd.set_option('display.max_columns', None)


In [21]:
player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='')
player_stats_df.head()


,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Suter,STL,D,17,324.666667,1,3,1,2,4,21.05,12,8.33,0.63,38,21,9,1,3,4,12,6,6,0,0,1,18,2,8,13,20,0,0,-
1,Brent Burns,CAR,D,15,241.783333,0,5,2,3,5,35.71,24,0.00,0.97,58,36,16,2,1,4,4,2,2,0,0,1,21,6,2,7,12,0,0,-
2,Corey Perry,EDM,R,17,173.516667,3,0,0,0,3,42.86,15,20.00,1.98,33,23,20,9,0,1,14,4,2,2,0,6,6,3,4,7,4,0,2,0.00
3,Alex Ovechkin,WSH,L,15,180.000000,7,7,4,3,14,77.78,28,25.00,2.81,58,40,34,14,0,5,2,1,1,0,0,0,5,4,37,7,1,0,0,-
4,Evgeni Malkin,PIT,C,18,275.583333,3,8,6,2,11,73.33,29,10.34,4.53,59,39,34,19,0,8,6,3,3,0,0,1,17,3,8,14,12,53,64,45.30


In [22]:
goalie_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='', pos='g')
goalie_stats_df.head()


,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,4,198.850000,92,85,7,0.924,2.11,1.16,5.51,12,9,3,0.750,0.91,-0.78,23,19,4,0.826,1.21,-1.44,51,51,0,1.000,0.00,1.74,6,10,39.33,20.14
1,Jonathan Quick,NYR,4,170.766667,87,86,1,0.989,0.35,6.72,6.76,22,22,0,1.000,0.00,4.06,26,25,1,0.962,0.35,1.89,37,37,0,1.000,0.00,1.26,6,15,33.29,30.00
2,James Reimer,ANA,2,103.600000,53,48,5,0.906,2.90,-0.30,5.20,7,4,3,0.571,1.74,-1.71,23,21,2,0.913,1.16,0.56,23,23,0,1.000,0.00,0.78,3,8,30.81,17.20
3,Semyon Varlamov,NYI,7,351.650000,152,138,14,0.908,2.39,-0.52,13.49,41,35,6,0.854,1.02,1.57,41,36,5,0.878,0.85,-0.44,55,52,3,0.945,0.51,-1.13,13,24,35.97,20.21
4,Jacob Markstrom,N.J,13,636.716667,298,271,27,0.909,2.54,-0.57,24.92,67,50,17,0.746,1.60,-4.63,80,73,7,0.913,0.66,1.90,132,129,3,0.977,0.28,1.49,38,57,39.48,19.67


In [23]:
class Lineup:
    def __init__(self, name):
        self.name = name
        self.forwards = [None] * 12  # 12 Forward Slots
        self.defense = [None] * 6    # 6 Defense Slots
        self.goalies = [None] * 2    # 2 Goalie Slots

    def add_forward(self, player, slot):
        allowed_positions = {'L', 'C', 'R'}
        if player.position not in allowed_positions:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position}' to forwards. "
                f"Allowed positions: {', '.join(allowed_positions)}."
            )
        if 0 <= slot < 12:
            if self.forwards[slot] is not None:
                print(f"Warning: Slot {slot + 1} in forwards is already occupied by '{self.forwards[slot].name}'. Overwriting.")
            self.forwards[slot] = player
            print(f"Added player '{player.name}' to forward slot {slot + 1}.")
        else:
            raise IndexError("Forward slot must be between 0 and 11.")

    def add_defense(self, player, slot):
        allowed_position = 'D'
        if player.position != allowed_position:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position}' to defense. "
                f"Allowed position: '{allowed_position}'."
            )
        if 0 <= slot < 6:
            if self.defense[slot] is not None:
                print(f"Warning: Slot {slot + 1} in defense is already occupied by '{self.defense[slot].name}'. Overwriting.")
            self.defense[slot] = player
            print(f"Added player '{player.name}' to defense slot {slot + 1}.")
        else:
            raise IndexError("Defense slot must be between 0 and 5.")

    def set_goalie(self, player, slot):
        allowed_position = 'G'
        if player.position != allowed_position:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position}' to goalies. "
                f"Allowed position: '{allowed_position}'."
            )
        if 0 <= slot < 2:
            if self.goalies[slot] is not None:
                print(f"Warning: Goalie slot {slot + 1} is already occupied by '{self.goalies[slot].name}'. Overwriting.")
            self.goalies[slot] = player
            print(f"Added goalie '{player.name}' to goalie slot {slot + 1}.")
        else:
            print(f"Cannot assign goalie to slot {slot}. Maximum of two goalies allowed.")

    def display_lineup(self):
        print(f"Lineup: {self.name}")
        print("Forwards:")
        for idx, player in enumerate(self.forwards):
            print(f"  Slot {idx + 1}: {player if player else 'Empty'}")
        print("Defense:")
        for idx, player in enumerate(self.defense):
            print(f"  Slot {idx + 1}: {player if player else 'Empty'}")
        print("Goalies:")
        for idx, player in enumerate(self.goalies):
            print(f"  Slot {idx + 1}: {player if player else 'Empty'}")
    def to_dataframe(self):
        """
        Converts the lineup into a pandas DataFrame.

        Returns:
            pd.DataFrame: A DataFrame containing forwards, defensemen, and goalies with their respective slots.
        """
        import pandas as pd

        # Prepare data for forwards
        forwards_data = [{
            'Position': 'F',
            'Slot': idx + 1,
            'Player': player.name if player else 'Empty'
        } for idx, player in enumerate(self.forwards)]

        # Prepare data for defensemen
        defense_data = [{
            'Position': 'D',
            'Slot': idx + 1,
            'Player': player.name if player else 'Empty'
        } for idx, player in enumerate(self.defense)]

        # Prepare data for goalies
        goalies_data = [{
            'Position': 'G',
            'Slot': idx + 1,
            'Player': player.name if player else 'Empty'
        } for idx, player in enumerate(self.goalies)]

        # Combine all data
        combined_data = forwards_data + defense_data + goalies_data

        # Create DataFrame
        df = pd.DataFrame(combined_data)

        return df
    def to_transposed_dataframe(self):
        """
        Transposes the lineup DataFrame so that each column represents a position-slot combination
        (e.g., 'Forward1', 'Defense1', 'Goalie1') and the row contains the corresponding player names.

        Returns:
            pd.DataFrame: A transposed DataFrame with position-slot as columns and player names as row values.
        """
        import pandas as pd

        # Generate the original DataFrame using the existing method
        df = self.to_dataframe()

        # Initialize an empty dictionary to hold the transposed data
        transposed_data = {}

        # Iterate over each row in the original DataFrame
        for _, row in df.iterrows():
            position = row['Position']
            slot = row['Slot']
            player = row['Player']
            # Create a new column name by combining position and slot number
            column_name = f"{position}{slot}"
            transposed_data[column_name] = player

        # Create a new DataFrame with a single row using the transposed data
        transposed_df = pd.DataFrame([transposed_data])

        return transposed_df

In [24]:
# Define the Player class
class Player:
    def __init__(self, name, team, position, player_id=None):
        """
        Initializes a new Player instance.

        :param name: Full name of the player
        :param team: Team the player belongs to
        :param position: Position of the player (e.g., C, LW, RW, D, G)
        :param player_id: Unique identifier for the player (optional)
        """
        self.player_id = player_id
        self.name = name
        self.team = team
        self.position = position

    def __str__(self):
        """
        Returns a string representation of the player.
        """
        return f"{self.name} ({self.position}) - {self.team}"

    def to_dict(self):
        """
        Converts the Player instance into a dictionary.
        """
        return {
            'player_id': self.player_id,
            'name': self.name,
            'team': self.team,
            'position': self.position
        }


In [25]:
# Creating Player instances from the player_stats_df DataFrame
player_list = []
for _, row in player_stats_df.iterrows():
    player = Player(
        name=row['player'],
        team=row['team'],
        position=row['position']
        # player_id is not set initially
    )
    player_list.append(player)

    # Convert player list to DataFrame
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position
} for player in player_list]).head()

,name,team,position
0,Ryan Suter,STL,D
1,Brent Burns,CAR,D
2,Corey Perry,EDM,R
3,Alex Ovechkin,WSH,L
4,Evgeni Malkin,PIT,C


In [26]:
# Creating Player instances from the player_stats_df DataFrame
goalie_list = []
for _, row in goalie_stats_df.iterrows():
    player = Player(
        name=row['player'],
        team=row['team'],
        position='G'
        # player_id is not set initially
    )
    goalie_list.append(player)

    # Convert player list to DataFrame
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position
} for player in goalie_list]).head()

,name,team,position
0,Marc-Andre Fleury,MIN,G
1,Jonathan Quick,NYR,G
2,James Reimer,ANA,G
3,Semyon Varlamov,NYI,G
4,Jacob Markstrom,N.J,G


In [27]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Construct the database configuration dictionary
db_config = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}


In [28]:
today_datetime= datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1, hours=6) # UTC offset
yesterday = yesterday_datetime.strftime('%Y-%m-%d')
yesterday

'2024-11-14'

In [ ]:
## TODO write function which updates the players database

In [11]:
append_player_ids(player_list, db_config)


Assigned player_id 8470600 to Ryan Suter.
Assigned player_id 8470613 to Brent Burns.
Assigned player_id 8470621 to Corey Perry.
Assigned player_id 8471214 to Alex Ovechkin.
Assigned player_id 8471215 to Evgeni Malkin.
Assigned player_id 8471675 to Sidney Crosby.
Assigned player_id 8471677 to Jack Johnson.
Assigned player_id 8471685 to Anze Kopitar.
Assigned player_id 8471724 to Kris Letang.
Assigned player_id 8471817 to Ryan Reaves.
Assigned player_id 8473419 to Brad Marchand.
Assigned player_id 8473422 to Nick Foligno.
Assigned player_id 8473446 to Erik Johnson.
Assigned player_id 8473453 to Trevor Lewis.
Assigned player_id 8473507 to Jeff Petry.
Assigned player_id 8473512 to Claude Giroux.
Assigned player_id 8473533 to Jordan Staal.
Assigned player_id 8473986 to Alex Killorn.
Assigned player_id 8473994 to Jamie Benn.
Assigned player_id 8474013 to Ian Cole.
Assigned player_id 8474034 to Patrick Maroon.
HTTP error occurred while fetching player data: 404 Client Error: Not Found for url

In [60]:
def create_lineup(team):
    """
    Creates and displays a lineup consisting of players from the specified team.
    
    Args:
        team (str): The team name to filter players.
    """
    # Creating two lineup objects
    lineup1 = Lineup("Lineup 1")
    
    # Adding forwards to lineup1
    forward_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_forward(player, forward_count)
                forward_count += 1
                if forward_count >= 12:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding defense to lineup1
    defense_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_defense(player, defense_count)
                defense_count += 1
                if defense_count >= 6:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding goalies to lineup1
    goalie_count = 0
    for goalie in goalie_list:
        if goalie.team != team:
            continue  # Proceed to the next goalie if not in the specified team
        if goalie_count >= 2:
            print("Maximum of two goalies have been assigned.")
            break
        try:
            lineup1.set_goalie(goalie, goalie_count)
            goalie_count += 1
        except ValueError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
        except IndexError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
    
    # Display the lineup
    # lineup1.display_lineup()
    return lineup1

my_lineup = create_lineup('TOR')

Skipping player 'Mark Giordano': Cannot add player 'Mark Giordano' with position 'D' to forwards. Allowed positions: L, R, C.
Added player 'Ryan Reaves' to forward slot 1.
Skipping player 'TJ Brodie': Cannot add player 'TJ Brodie' with position 'D' to forwards. Allowed positions: L, R, C.
Added player 'John Tavares' to forward slot 2.
Added player 'Calle Jarnkrok' to forward slot 3.
Skipping player 'John Klingberg': Cannot add player 'John Klingberg' with position 'D' to forwards. Allowed positions: L, R, C.
Skipping player 'Morgan Rielly': Cannot add player 'Morgan Rielly' with position 'D' to forwards. Allowed positions: L, R, C.
Skipping player 'Jake McCabe': Cannot add player 'Jake McCabe' with position 'D' to forwards. Allowed positions: L, R, C.
Added player 'Tyler Bertuzzi' to forward slot 4.
Added player 'Max Domi' to forward slot 5.
Added player 'William Nylander' to forward slot 6.
Added player 'Mitch Marner' to forward slot 7.
Added player 'Auston Matthews' to forward slot 8

In [61]:
# Convert the lineup to a transposed DataFrame
transposed_lineup_df = my_lineup.to_transposed_dataframe()

# Display the transposed DataFrame
transposed_lineup_df

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,D1,D2,D3,D4,D5,D6,G1,G2
0,Ryan Reaves,John Tavares,Calle Jarnkrok,Tyler Bertuzzi,Max Domi,William Nylander,Mitch Marner,Auston Matthews,Noah Gregor,David Kampf,Pontus Holmberg,Nicholas Robertson,Mark Giordano,TJ Brodie,John Klingberg,Morgan Rielly,Jake McCabe,Maxime Lajoie,Martin Jones,Ilya Samsonov


In [10]:
from pbp import get_team_roster

get_team_roster('TOR', 20232024)

{'forwards': [{'id': 8481720,
   'headshot': 'https://assets.nhle.com/mugs/nhl/20232024/TOR/8481720.png',
   'firstName': {'default': 'Nick',
    'cs': 'Nicholas',
    'de': 'Nicholas',
    'es': 'Nicholas',
    'fi': 'Nicholas',
    'sk': 'Nicholas',
    'sv': 'Nicholas'},
   'lastName': {'default': 'Abruzzese'},
   'sweaterNumber': 26,
   'positionCode': 'C',
   'shootsCatches': 'L',
   'heightInInches': 71,
   'weightInPounds': 180,
   'heightInCentimeters': 180,
   'weightInKilograms': 82,
   'birthDate': '1999-06-04',
   'birthCity': {'default': 'Slate Hill'},
   'birthCountry': 'USA',
   'birthStateProvince': {'default': 'NY'}},
  {'id': 8480980,
   'headshot': 'https://assets.nhle.com/mugs/nhl/20232024/TOR/8480980.png',
   'firstName': {'default': 'Connor'},
   'lastName': {'default': 'Dewar'},
   'sweaterNumber': 24,
   'positionCode': 'C',
   'shootsCatches': 'L',
   'heightInInches': 70,
   'weightInPounds': 183,
   'heightInCentimeters': 178,
   'weightInKilograms': 83,
   '

In [13]:
from pbp_utils import get_matchup_games

games = get_matchup_games('2024-11-11', '2024-11-17')
pd.DataFrame({
    'game_id': games['game_ids']['id'],
    'date': games['game_ids']['date']
}).head()


,game_id,date
0,2024020239,2024-11-11
1,2024020240,2024-11-11
2,2024020241,2024-11-11
3,2024020242,2024-11-11
4,2024020243,2024-11-11


In [8]:
from game_utils import get_game_boxscore, display_boxscore

temp_data = get_game_boxscore(2024020255)

In [9]:
away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(temp_data)
away_skaters


,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8477960,9,A. Kempe,R,2,0,2,0,0,1,0,4,0.000000,19:11,1,23,0,0,Away
1,8471685,11,A. Kopitar,C,0,2,2,0,0,0,0,1,0.769231,19:29,0,23,0,0,Away
2,8479675,12,T. Moore,L,0,2,2,0,0,1,0,0,0.000000,18:52,1,24,0,0,Away
3,8482155,14,A. Laferriere,R,0,0,0,-3,0,2,0,1,0.000000,15:46,0,22,0,1,Away
4,8477942,22,K. Fiala,L,0,0,0,-1,2,2,0,1,0.000000,18:00,2,22,2,0,Away
5,8476479,24,P. Danault,C,0,0,0,-1,2,1,0,0,0.636364,18:11,1,25,1,1,Away
6,8480851,26,A. Thomas,C,0,0,0,0,0,1,0,0,0.000000,11:09,1,15,0,0,Away
7,8477998,37,W. Foegele,L,0,0,0,-1,0,2,0,1,0.000000,15:37,1,25,2,0,Away
8,8481732,47,A. Lee,L,0,0,0,-1,0,3,0,2,0.000000,10:43,2,13,0,1,Away
9,8482124,55,Q. Byfield,R,0,0,0,-1,0,1,0,1,0.500000,13:46,0,22,2,0,Away


In [10]:
away_goalies


,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,team
0,8479496,31,D. Rittich,G,4/5,0/0,0/0,4/5,0.8,1,0,0,0,1,14:41,False,L,5,4,Away
1,8475311,35,D. Kuemper,G,17/19,1/1,0/0,18/20,0.9,2,0,0,0,2,44:32,True,NaN,20,18,Away


In [11]:
home_skaters

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8476391,9,T. Tynan,C,0,0,0,0,0,0,0,0,0.000000,07:39,1,12,1,0,Home
1,8475780,14,C. Wagner,R,0,0,0,0,0,0,0,0,0.000000,04:24,0,10,0,0,Home
2,8480448,17,P. Kelly,C,0,0,0,0,0,3,0,1,0.555556,13:12,2,21,1,0,Home
3,8481186,25,L. O'Connor,R,0,1,1,1,0,1,0,0,0.000000,15:56,0,23,1,0,Home
4,8477492,29,N. MacKinnon,C,0,3,3,3,0,1,0,3,0.272727,24:45,0,27,1,0,Home
5,8479999,37,C. Mittelstadt,C,0,0,0,-1,0,0,0,1,0.428571,18:35,0,22,3,2,Home
6,8481042,51,N. Kovalenko,R,0,0,0,-1,2,2,0,0,0.000000,14:57,0,19,0,0,Home
7,8477476,62,A. Lehkonen,L,1,1,2,2,0,1,0,3,0.000000,23:13,1,25,0,0,Home
8,8483930,82,I. Ivan,C,0,0,0,-1,0,1,0,1,0.000000,16:18,1,19,0,0,Home
9,8485105,85,N. Prishchepov,C,0,0,0,0,0,1,0,1,0.000000,04:23,0,8,0,0,Home


In [14]:
home_goalies

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,team
0,8480382,40,A. Georgiev,G,11/13,2/2,0/0,13/15,0.866667,2,0,0,0,2,60:00,True,W,15,13,Home
1,8481020,60,J. Annunen,G,0/0,0/0,0/0,0/0,NaN,0,0,0,0,0,00:00,False,NaN,0,0,Home


In [17]:
lineup2.display_lineup()


Lineup: Lineup 2
Forwards:
  Slot 1: Empty
  Slot 2: Empty
  Slot 3: Empty
  Slot 4: Empty
  Slot 5: Empty
  Slot 6: Empty
  Slot 7: Empty
  Slot 8: Empty
  Slot 9: Empty
  Slot 10: Empty
  Slot 11: Empty
  Slot 12: Empty
Defense:
  Slot 1: Empty
  Slot 2: Empty
  Slot 3: Empty
  Slot 4: Empty
  Slot 5: Empty
  Slot 6: Empty
Goalie: Empty

